In [1]:
from IPython.display import HTML

# Explorative analysis and data visualization
#### Roman Jurowetzki - SDS 2018, S3-4 - 7 September 2018

This is a Jupyter notebook which you are going to see and use a lot as an aspiring data scientist. Notebooks can be run with different kernels (Python, R, F# etc.). This one runs Python 3. 

Notebooks combine Markdown cells for content with computational cells. This allows to 

In this session we will explore a **real** dataset from the Stanford Open Policing Project. 
The project is collecting and standardizing data on vehicle and pedestrian stops from law enforcement departments across the US — and making that information freely available. They have already gathered 130 million records from 31 state police agencies and have begun collecting data on stops from law enforcement agencies in major cities, as well.

You can read more about the project [here](https://openpolicing.stanford.edu)

![open police](https://comm.stanford.edu/mm/2017/08/open-policing-project.jpg)

In [18]:
HTML('<iframe width="800" height="500" src="https://www.youtube-nocookie.com/embed/PelSGxTPlXM?rel=0&amp;controls=0&amp;showinfo=0&amp;start=435" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>')

This session will also be an introduction to the Python data science stack, specifically the __pandas__ and __seaborn__ packages. 

![python_stack](http://chris35wills.github.io/courses/pydata_stack.png)

- Pandas is Python's main library for managing and analysing dataframes. 
- Seaborn is a high-level library for statistical visualisation. 

Everything that we do today can be easily transleted in R (using dplys, ggplot etc.) Thus, it is more about the concepts than the particular language.

---

## Agenda

- Import and examine the data for one US state
- Preprocess the data (cleaning, adjusting datatypes)
- Calculate some simple statistics on different levels of aggregation
- Make informative plots
- Come up and *hypotheses*

## Let's get started

First we'll download the latest dataset directly from the Stanford server. To do that we use Jupyter's **!** command line magic. Passing a `!` in a code cell, will send the command to shell rather than Python or R. `!wget` will open the GNU Wget Unix/Linux program that downloads content from web servers.

In [37]:
# Download the semi-raw data
!wget https://stacks.stanford.edu/file/druid:py883nd2578/RI-clean.csv.gz

--2018-08-31 09:48:30--  https://stacks.stanford.edu/file/druid:py883nd2578/RI-clean.csv.gz
Resolving stacks.stanford.edu (stacks.stanford.edu)... 171.67.37.91
Connecting to stacks.stanford.edu (stacks.stanford.edu)|171.67.37.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6026983 (5.7M) [application/gzip]
Saving to: ‘RI-clean.csv.gz’

RI-clean.csv.gz     100%[===================>]   5.75M   365KB/s    in 17s     

2018-08-31 09:48:48 (349 KB/s) - ‘RI-clean.csv.gz’ saved [6026983/6026983]



In [39]:
# Unzip the file to get a csv
!gunzip RI-clean.csv.gz

In [135]:
!ls -lh

total 1.1G
-rw-r--r-- 1 roman@id.aau.dk 105801 1017M Jun 18  2017 IL-clean.csv
-rw-r--r-- 1 roman@id.aau.dk 105801   75M Jun 18  2017 RI-clean.csv
-rw-r--r-- 1 roman@id.aau.dk 105801   44K Aug 31 14:24 Untitled.ipynb


Let's import pandas as pd. The appbriviation pd is a convention. We also load the data using pandas's `read_csv` command. Pandas will try to infer the appropriate datatypes dor all columns. We set the `low_memory` argument to `False`, which is often done in cases with possible mixed datatypes. By the way: You if you place the cursor after the '(' after a command and use string+tab, you can read the docummentation for the particular command or function.

In [89]:
import pandas as pd

In [136]:
data = pd.read_csv('RI-clean.csv', low_memory=False)

We can examine the dataset in several ways, for instance, by checking the first couple of rows or by printing the "info".

In [137]:
#Display the first 5 (or more/less rows) of each column
data.head()

,id,state,stop_date,stop_time,location_raw,county_name,county_fips,fine_grained_location,police_department,driver_gender,...,search_conducted,search_type_raw,search_type,contraband_found,stop_outcome,is_arrested,stop_duration,out_of_state,drugs_related_stop,district
0,RI-2005-00001,RI,2005-01-02,01:55,Zone K1,NaN,NaN,NaN,600,M,...,False,NaN,NaN,False,Citation,False,0-15 Min,False,False,Zone K1
1,RI-2005-00002,RI,2005-01-02,20:30,Zone X4,NaN,NaN,NaN,500,M,...,False,NaN,NaN,False,Citation,False,16-30 Min,False,False,Zone X4
2,RI-2005-00003,RI,2005-01-04,11:30,Zone X1,NaN,NaN,NaN,000,NaN,...,False,NaN,NaN,False,NaN,NaN,NaN,NaN,False,Zone X1
3,RI-2005-00004,RI,2005-01-04,12:55,Zone X4,NaN,NaN,NaN,500,M,...,False,NaN,NaN,False,Citation,False,0-15 Min,False,False,Zone X4
4,RI-2005-00005,RI,2005-01-06,01:30,Zone X4,NaN,NaN,NaN,500,M,...,False,NaN,NaN,False,Citation,False,0-15 Min,False,False,Zone X4


- Each row is one traffic stop
- NaN are missing values

In [138]:
#Display overview information for the dataframe
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 509681 entries, 0 to 509680
Data columns (total 26 columns):
id                       509681 non-null object
state                    509681 non-null object
stop_date                509671 non-null object
stop_time                509671 non-null object
location_raw             509681 non-null object
county_name              0 non-null float64
county_fips              0 non-null float64
fine_grained_location    0 non-null float64
police_department        509671 non-null object
driver_gender            480584 non-null object
driver_age_raw           480632 non-null float64
driver_age               478986 non-null float64
driver_race_raw          480608 non-null object
driver_race              480608 non-null object
violation_raw            480608 non-null object
violation                480608 non-null object
search_conducted         509671 non-null object
search_type_raw          17762 non-null object
search_type              17762 non-n

In [139]:
# print the shape of the DF as an easy alternative to looking up this in info.
data.shape

(509681, 26)

### Missing values

In [140]:
# Show missing values (mask), cut to the first 10 rows. Note, that python index alsways starts with 0!
data.isnull()[:10]

,id,state,stop_date,stop_time,location_raw,county_name,county_fips,fine_grained_location,police_department,driver_gender,...,search_conducted,search_type_raw,search_type,contraband_found,stop_outcome,is_arrested,stop_duration,out_of_state,drugs_related_stop,district
0,False,False,False,False,False,True,True,True,False,False,...,False,True,True,False,False,False,False,False,False,False
1,False,False,False,False,False,True,True,True,False,False,...,False,True,True,False,False,False,False,False,False,False
2,False,False,False,False,False,True,True,True,False,True,...,False,True,True,False,True,True,True,True,False,False
3,False,False,False,False,False,True,True,True,False,False,...,False,True,True,False,False,False,False,False,False,False
4,False,False,False,False,False,True,True,True,False,False,...,False,True,True,False,False,False,False,False,False,False
5,False,False,False,False,False,True,True,True,False,False,...,False,True,True,False,False,False,False,False,False,False
6,False,False,False,False,False,True,True,True,False,False,...,False,True,True,False,False,False,False,False,False,False
7,False,False,False,False,False,True,True,True,False,False,...,False,True,True,False,False,False,False,False,False,False
8,False,False,False,False,False,True,True,True,False,False,...,False,True,True,False,False,False,False,False,False,False
9,False,False,False,False,False,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [141]:
# The isnull command generates a dataframe with bool (True/False) outputs that you can apply commands on.
data.isnull().sum()

id                            0
state                         0
stop_date                    10
stop_time                    10
location_raw                  0
county_name              509681
county_fips              509681
fine_grained_location    509681
police_department            10
driver_gender             29097
driver_age_raw            29049
driver_age                30695
driver_race_raw           29073
driver_race               29073
violation_raw             29073
violation                 29073
search_conducted             10
search_type_raw          491919
search_type              491919
contraband_found              0
stop_outcome              29073
is_arrested               29073
stop_duration             29073
out_of_state              29881
drugs_related_stop            0
district                      0
dtype: int64

We can see that the county_name column is all missing. And thus we can drop it. Also, we probably should drop all cases, where we don't know the time and date of the stop, the gender of the driver

In [142]:
# We drop the fine_grained_location, county_fips, county_name columns.
data.drop(['county_name', 'county_fips', 'fine_grained_location'], axis='columns', inplace=True)

In [143]:
# Drop all columns where stop_data, time, and driver_gender are missing
data.dropna(subset=['stop_date', 'stop_time', 'driver_gender'], inplace=True)

In [144]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 480584 entries, 0 to 509670
Data columns (total 23 columns):
id                    480584 non-null object
state                 480584 non-null object
stop_date             480584 non-null object
stop_time             480584 non-null object
location_raw          480584 non-null object
police_department     480584 non-null object
driver_gender         480584 non-null object
driver_age_raw        480583 non-null float64
driver_age            478946 non-null float64
driver_race_raw       480584 non-null object
driver_race           480584 non-null object
violation_raw         480584 non-null object
violation             480584 non-null object
search_conducted      480584 non-null object
search_type_raw       17762 non-null object
search_type           17762 non-null object
contraband_found      480584 non-null bool
stop_outcome          480584 non-null object
is_arrested           480584 non-null object
stop_duration         480584 non-nul

### Adjusting data types

In [145]:
# let's check this potential candidate column
data.is_arrested.head()

0    False
1    False
3    False
4    False
5    False
Name: is_arrested, dtype: object

This column should be boolian but for some reason ended up being a string (or object). This is not efficient and limits our options in terms of what we can do with it. Therefore, we will change the datatype.

#### On bracket vs. dot notation:

In Python you will find 2 notation types. [ ] and .

``` ri['is_arrested'] is the same as ri.is_arrested````

However, if you assign something and have it on the left side of = you should always use [ ] notation



In [146]:
# Assign the is_arrested column a new datatype
data['is_arrested'] = data.is_arrested.astype('bool')

Is there another column that may suffer from this problem?

### Adjusting dates, times and index

As you can see, stop_date and time are objects. That's not very useful. Let's transform them into a handy date-time-index.

First, we will concatenate the two columns into one. Second, we will ask pandas to parse it and set the DF's index as the date and time of the stop. This makes lots of sense, given that each row is an *event*.

In [147]:
# We start by concatenating the two string columns into one that we call combined, using str.cat

combined = data.stop_date.str.cat(data.stop_time, sep=' ')
print(combined.head())

0    2005-01-02 01:55
1    2005-01-02 20:30
3    2005-01-04 12:55
4    2005-01-06 01:30
5    2005-01-12 08:05
Name: stop_date, dtype: object


Pandas provides many really nice string options that you definetely should explore. Just set your cursor after str and press tab for a list of options.

In [148]:
# You can for example get dummies from a categorical string variable (here just for the first 10)
data.violation[:10].str.get_dummies()

,Equipment,Moving violation,Other,Registration/plates,Speeding
0,0,0,0,0,1
1,0,0,0,0,1
3,1,0,0,0,0
4,1,0,0,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,1,0,0,0
8,0,0,0,0,1
9,0,0,0,0,1
10,0,0,0,1,0


Now we can create a date_and_time column form our combined (Series - basically a DF with only one column). Finally we will set the index of the dateframe to be the column (instead of a normal index). This will open up for many options, e.g. resampling.

In [149]:
# Parse the combined series to datetime-format and assign it to a new column
data['date_and_time'] = pd.to_datetime(combined)

In [150]:
# Replace the standard index by the new column (which will in turn disappear)
data.set_index('date_and_time', inplace=True)

In [151]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 480584 entries, 2005-01-02 01:55:00 to 2015-12-31 23:48:00
Data columns (total 23 columns):
id                    480584 non-null object
state                 480584 non-null object
stop_date             480584 non-null object
stop_time             480584 non-null object
location_raw          480584 non-null object
police_department     480584 non-null object
driver_gender         480584 non-null object
driver_age_raw        480583 non-null float64
driver_age            478946 non-null float64
driver_race_raw       480584 non-null object
driver_race           480584 non-null object
violation_raw         480584 non-null object
violation             480584 non-null object
search_conducted      480584 non-null object
search_type_raw       17762 non-null object
search_type           17762 non-null object
contraband_found      480584 non-null bool
stop_outcome          480584 non-null object
is_arrested           480584 non-null bool
stop

## Exploring the data step by step

We will start with simple countrs, proportions, averages etc. and move from there to more advanced concepts

In [154]:
# We can explore unique values for a column (even if it's a string)
data.stop_outcome.unique()

array(['Citation', 'Arrest Driver', 'N/D', 'Warning', 'Arrest Passenger',
       'No Action'], dtype=object)

In [161]:
# count the distinct values
data.stop_outcome.value_counts()

CPU times: user 56 ms, sys: 0 ns, total: 56 ms
Wall time: 57.4 ms


Citation            428378
Warning              28840
Arrest Driver        14630
N/D                   3431
No Action             3332
Arrest Passenger      1973
Name: stop_outcome, dtype: int64

In [162]:
# there are many ways to do the same thing
data.groupby('stop_outcome').size()

CPU times: user 64 ms, sys: 0 ns, total: 64 ms
Wall time: 30.2 ms


stop_outcome
Arrest Driver        14630
Arrest Passenger      1973
Citation            428378
N/D                   3431
No Action             3332
Warning              28840
dtype: int64

In [163]:
# Value counts provides a nice proportions option
data.stop_outcome.value_counts(normalize=True)

Citation            0.891370
Warning             0.060010
Arrest Driver       0.030442
N/D                 0.007139
No Action           0.006933
Arrest Passenger    0.004105
Name: stop_outcome, dtype: float64

In [170]:
# Let's check distribution by race
data.driver_race.value_counts(normalize=True)

White       0.717286
Black       0.142695
Hispanic    0.110538
Asian       0.026684
Other       0.002797
Name: driver_race, dtype: float64

### Let's try out some hypotheses

One hypothesis could be that the stop_outcome is different for different races. Discrimination?

In [173]:
black = data[data.driver_race == 'Black']
white = data[data.driver_race == 'White']
hispanic = data[data.driver_race == 'Hispanic']

In [174]:
black.stop_outcome.value_counts(normalize=True)

Citation            0.858495
Warning             0.067953
Arrest Driver       0.049273
N/D                 0.009085
Arrest Passenger    0.008151
No Action           0.007043
Name: stop_outcome, dtype: float64

In [175]:
white.stop_outcome.value_counts(normalize=True)

Citation            0.902485
Warning             0.057427
Arrest Driver       0.023814
No Action           0.006724
N/D                 0.006568
Arrest Passenger    0.002982
Name: stop_outcome, dtype: float64

In [176]:
hispanic.stop_outcome.value_counts(normalize=True)

Citation            0.852211
Warning             0.071005
Arrest Driver       0.052708
N/D                 0.008998
No Action           0.008377
Arrest Passenger    0.006701
Name: stop_outcome, dtype: float64

#### Let's try out to come up with some interesting hypotheses and find answers using the methods that we learned so far.

### Filtering my multiple conditions

We can of cause chain filter conditions (you probably learned to do that in tha last session using R)

In [181]:
hispanic_and_arrested = data[(data.driver_race == 'Hispanic')
                             & (data.is_arrested == True)]

In [184]:
len(hispanic_and_arrested)

3156

In [186]:
hispanic_or_arrested = data[(data.driver_race == 'Hispanic')
                             | (data.is_arrested == True)]

In [187]:
hispanic_or_arrested.shape

(66570, 24)

##### Rules for filtering

- & AND
- | OR
- Each condition must be surrounded by () and many are possible
- == Equality
- != Inequality

Remember, that we are not making any statement about causation. This is purely a correlation exercise (so far!)

Sometimes during stops if a search is conducted, the officer also checks the driver if they have a weapon. This is called a "protective frisk".
Let's try to figure out if men are frisked more than women.

In [177]:
data.search_type.value_counts()

Incident to Arrest                                          6998
Probable Cause                                              4989
Reasonable Suspicion                                        1141
Inventory                                                   1101
Protective Frisk                                             879
Incident to Arrest,Inventory                                 649
Incident to Arrest,Probable Cause                            552
Probable Cause,Reasonable Suspicion                          334
Probable Cause,Protective Frisk                              221
Incident to Arrest,Protective Frisk                          158
Incident to Arrest,Inventory,Probable Cause                  151
Inventory,Probable Cause                                     132
Protective Frisk,Reasonable Suspicion                         83
Incident to Arrest,Inventory,Protective Frisk                 77
Incident to Arrest,Probable Cause,Protective Frisk            74
Inventory,Protective Fris

In [178]:
# We ask pandas to find out if the string in the search_type column contatins
# the sequence "Protective Frisk"
# We assign the result to a new column that we call "frisk"

data['frisk'] = data.search_type.str.contains('Protective Frisk', na = False)

In [180]:
data['frisk'].sum()

1598